# 5. Ensembles de Arboles de Decision

## 5.4 GBDT LightGBM

La técnica de Gradient Boosting fue creada por Jerome H. Friedman en 1999 - 2001
<br>Se implementaron librerías ineficientes
<br>En 2016 se crea XGBoost, en 2017 LightGBM

El Gradient Boosting of Decision Trees es un ensemble de árboles de decisión, para un nuevo registro la predicción se hace sumando el score que cada arbol asigna a ese registro.

En GBDT la construccion de los árboles es secuencial, ya que el arbol n-simo se genera para predecir el error del modelo conformado por los  n-1 arboles previos, aunque sea un arbol de clasificación lo que se predice es un numero real mediante un arbol de regresión.


<br>Qué tipo de perturbaciones se realiza LightGBM

*   Se perturba el dataset, seleccionando para cada arbol un subconjunto de las columnas.
*   El algortimo de arbol de decisión no presenta perturbaciones

Cada arbolito de LightGBM se entrena sobre un dataset perturbado, que en principio posee :
* todos los registros del dataset original
* solo un porcentaje *feature_fraction* de las columnas originales del dataset

#### 5.4.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 5.4.2  LightGBM, una corrida

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [123]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1819486,97.2,2898550,154.8,2898550,154.8
Vcells,3568860,27.3,118236152,902.1,139061623,1061.0


In [124]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

Aqui debe cargar SU semilla primigenia

In [125]:
PARAM <- list()
PARAM$experimento <- 5420
PARAM$semilla_primigenia <- 100103

# estos hiperparametros de LightGBM surgieron de una Bayesian Optimization
PARAM$lgb$num_iterations <- 2500  # cantidad de arbolitos
PARAM$lgb$learning_rate <- 0.027
PARAM$lgb$feature_fraction <- 0.8
PARAM$lgb$min_data_in_leaf <- 76
PARAM$lgb$num_leaves <- 8
PARAM$lgb$max_bin <- 31


In [126]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KA_leaves2", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [127]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv", stringsAsFactors = TRUE)

In [128]:
# paso la clase a binaria que tome valores {0,1}  enteros
# set trabaja con la clase  POS = { BAJA+1, BAJA+2 }
# esta estrategia es MUY importante
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2", "BAJA+1"), 1L, 0L)]

In [129]:
# los campos que se van a utilizar
campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01"))

In [130]:
# establezco donde entreno
dataset[, train := 0L]
dataset[foto_mes %in% c(202107), train := 1L]

In [131]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[train == 1L, campos_buenos, with = FALSE]),
  label= dataset[train == 1L, clase01]
)

In [132]:
# genero el modelo
# estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
set.seed( PARAM$semilla_primigenia ) # Establezco la semilla aleatoria

modelo <- lgb.train(
  data= dtrain,
  param= list(
    objective= "binary",
    max_bin= PARAM$lgb$max_bin,
    learning_rate= PARAM$lgb$learning_rate,
    num_iterations= PARAM$lgb$num_iterations,
    num_leaves= PARAM$lgb$num_leaves,
    min_data_in_leaf= PARAM$lgb$min_data_in_leaf,
    feature_fraction= PARAM$lgb$feature_fraction,
    seed= PARAM$semilla_primigenia
  )
)


[LightGBM] [Info] Number of positive: 2385, number of negative: 162211
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038448 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3612
[LightGBM] [Info] Number of data points in the train set: 164596, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014490 -> initscore=-4.219699
[LightGBM] [Info] Start training from score -4.219699


In [133]:
# ahora imprimo la importancia de variables
tb_importancia <- as.data.table(lgb.importance(modelo))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file = archivo_importancia,
  sep = "\t"
)


In [134]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo, "modelo.txt" )

In [135]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo,
  data.matrix(dfuture[, campos_buenos, with = FALSE])
)


In [136]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file = "prediccion.txt",
  sep = "\t"
)

In [137]:
# subidas a Kaggle
# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

In [138]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

cortes <- seq(9000, 13500, by = 500)

for (envios in cortes) {

  tb_prediccion[, Predicted := 0L]
  tb_prediccion[1:envios, Predicted := 1L]

  archivo_kaggle <- paste0("KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )

  # subida a Kaggle
  comando <- "kaggle competitions submit"
  competencia <- "-c labo-i-2025-rosario"
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'num_iterations=", PARAM$lgb$num_iterations,
    "  learning_rate=", PARAM$lgb$learning_rate,
    "  feature_fraction=", PARAM$lgb$feature_fraction,
    "  min_data_in_leaf=", PARAM$lgb$min_data_in_leaf,
    "  num_leaves=",PARAM$lgb$num_leaves,
    "  max_bin=", PARAM$lgb$max_bin,
  "'" )

  linea <- paste( comando, competencia, arch, mensaje)
  salida <- system(linea, intern=TRUE)
  cat(salida)
}

Successfully submitted to LaboI 2025 Rosario

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-rosario -f KA5420_9500.csv -m 'num_iterations=2500  learning_rate=0.027  feature_fraction=0.8  min_data_in_leaf=76  num_leaves=8  max_bin=31'' had status 1”


400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-rosario -f KA5420_10000.csv -m 'num_iterations=2500  learning_rate=0.027  feature_fraction=0.8  min_data_in_leaf=76  num_leaves=8  max_bin=31'' had status 1”


400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario

...

400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-rosario -f KA5420_13000.csv -m 'num_iterations=2500  learning_rate=0.027  feature_fraction=0.8  min_data_in_leaf=76  num_leaves=8  max_bin=31'' had status 1”


400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-rosario -f KA5420_13500.csv -m 'num_iterations=2500  learning_rate=0.027  feature_fraction=0.8  min_data_in_leaf=76  num_leaves=8  max_bin=31'' had status 1”


400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario



---



### 5.4.3  LightGBM  optimizacion de hiperparámetros

La optimizacion de los hiperparámetros de LightGBM mediante el método de optimizacion bayesiana será su *caballito de batalla* durante la asignatura !

limpio el ambiente de R

In [139]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1819891,97.2,4493198,240.0,4493198,240.0
Vcells,3569753,27.3,118919960,907.3,139963463,1067.9


In [140]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Loading required package: parallel

Loading required package: primes

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




Aqui debe cargar SU semilla primigenia

In [156]:
PARAM <- list()
PARAM$experimento <- 5431
PARAM$semilla_primigenia <- 100103

# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos
PARAM$trainingstrategy$undersampling <- 1.0

PARAM$hyperparametertuning$iteraciones <- 500
PARAM$hyperparametertuning$xval_folds <- 5
PARAM$hyperparametertuning$POS_ganancia <- 117000
PARAM$hyperparametertuning$NEG_ganancia <- -3000

# Aqui se cargan los bordes de los hiperparametros
PARAM$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.3),
  makeIntegerParam("num_leaves", lower = 8L, upper = 1024L),
  makeNumericParam("feature_fraction", lower = 0.1, upper = 1.0),
  makeIntegerParam("min_data_in_leaf", lower = 1L, upper = 8000L),
  makeIntegerParam("envios", lower = 5000L, upper = 15000L),
  makeNumericParam("lambda_l1", lower = 1e-8, upper = 10)
)

In [157]:
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(
    reg, arch = NA, folder = "./work/",
    ext = ".txt", verbose = TRUE) {

  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  if (!file.exists(archivo)) # Escribo los titulos
    {
      linea <- paste0(
        "fecha\t",
        paste(list.names(reg), collapse = "\t"), "\n"
      )

      cat(linea, file = archivo)
    }

  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t", # la fecha y hora
    gsub(", ", "\t", toString(reg)), "\n"
  )

  cat(linea, file = archivo, append = TRUE) # grabo al archivo

  if (verbose) cat(linea) # imprimo por pantalla
}


In [158]:
# esta funcion calcula internamente la ganancia de la prediccion probs
# es llamada por lightgbm luego de construir cada  arbolito

fganancia_logistic_lightgbm <- function(probs, datos) {
  vpesos <- get_field(datos, "weight")

  # vector de ganancias
  vgan <- ifelse(vpesos == 1.0000002, PARAM$hyperparametertuning$POS_ganancia,
    ifelse(vpesos == 1.0000001, PARAM$hyperparametertuning$NEG_ganancia,
      PARAM$hyperparametertuning$NEG_ganancia /
        PARAM$trainingstrategy$undersampling
    )
  )

  tbl <- as.data.table(list("vprobs" = probs, "vgan" = vgan))
  setorder(tbl, -vprobs)
  ganancia <- tbl[1:GLOBAL_envios, sum(vgan)]

  return(list(
    "name" = "ganancia",
    "value" = ganancia,
    "higher_better" = TRUE
  ))
}


In [159]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros se pasan como variables globales,
# la semilla del mal ...


EstimarGanancia_lightgbm <- function(x) {
  gc() # libero memoria

  # llevo el registro de la iteracion por la que voy
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # para usar en fganancia_logistic_lightgbm
  # asigno la variable global
  GLOBAL_envios <<- as.integer(x$envios / PARAM$hyperparametertuning$xval_folds)

  # cantidad de folds para cross validation
  kfolds <- PARAM$hyperparametertuning$xval_folds

  param_basicos <- list(
    objective= "binary",
    metric= "custom",
    first_metric_only= TRUE,
    boost_from_average= TRUE,
    feature_pre_filter= FALSE,
    verbosity= -100,
    max_bin= 31, # por ahora, lo dejo fijo
    num_iterations= 9999, # valor grande, lo limita early_stopping_rounds
    force_row_wise= TRUE, # para evitar warning
    seed= ksemilla_azar1
  )

  # el parametro discolo, que depende de otro
  param_variable <- list(
    early_stopping_rounds =
      as.integer(50 + 5 / x$learning_rate)
  )

  param_completo <- c(param_basicos, param_variable, x)

  set.seed(ksemilla_azar1)
  modelocv <- lgb.cv(
    data= dtrain,
    eval= fganancia_logistic_lightgbm,
    stratified= TRUE, # sobre el cross validation
    nfold= kfolds, # folds del cross validation
    param= param_completo,
    verbose= -100
  )

  # obtengo la ganancia
  ganancia <- unlist(modelocv$record_evals$valid$ganancia$eval)[modelocv$best_iter]

  ganancia_normalizada <- ganancia * kfolds # normailizo la ganancia

  # asigno el mejor num_iterations
  param_completo$num_iterations <- modelocv$best_iter
  # elimino de la lista el componente
  param_completo["early_stopping_rounds"] <- NULL


  # el lenguaje R permite asignarle ATRIBUTOS a cualquier variable
  # esta es la forma de devolver un parametro extra
  attr(ganancia_normalizada, "extras") <-
    list("num_iterations" = modelocv$best_iter)

  # logueo
  xx <- param_completo
  xx$ganancia <- ganancia_normalizada # le agrego la ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch = klog)

  # Voy registrando la importancia de variables
  if (ganancia_normalizada > GLOBAL_gananciamax) {
    GLOBAL_gananciamax <<- ganancia_normalizada
    modelo <- lgb.train(
      data = dtrain,
      param = param_completo,
      verbose = -100
    )

    tb_importancia <- as.data.table(lgb.importance(modelo))
    archivo_importancia <- paste0("impo_", GLOBAL_iteracion, ".txt")
    fwrite(tb_importancia,
      file = archivo_importancia,
      sep = "\t" )

    loguear(xx, arch = klog_mejor)
  }

  return(ganancia_normalizada)
}


aqui se inicia el programa

In [160]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [161]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, 2 )
ksemilla_azar1 <- PARAM$semillas[1]
ksemilla_azar2 <- PARAM$semillas[2]


In [162]:
# en estos archivos quedan los resultados

kbayesiana <- paste0(PARAM$experimento, ".RDATA")
klog <- paste0(PARAM$experimento, ".txt")
klog_mejor <- paste0(PARAM$experimento, "_mejor.txt")

GLOBAL_iteracion <- 0 # inicializo la variable global
GLOBAL_gananciamax <- -1 # inicializo la variable global

# si ya existe el archivo log, traigo hasta donde llegue
if (file.exists(klog)) {
  tabla_log <- fread(klog)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_gananciamax <- tabla_log[, max(ganancia)]
}


In [163]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [164]:
dataset <- dataset[foto_mes %in% c(202107)]

In [165]:
# paso la clase a binaria que tome valores {0,1}  enteros
dataset[
  foto_mes %in% c(202107),
  clase01 := ifelse(clase_ternaria == "CONTINUA", 0L, 1L)
]

In [166]:
# los campos que se van a utilizar
campos_buenos <- setdiff(
  colnames(dataset),
  c("clase_ternaria", "clase01", "azar", "training")
)


In [167]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(ksemilla_azar2)
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]
dataset[
  foto_mes %in% c(202107) &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [168]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label = dataset[training == 1L, clase01],
  weight = dataset[training == 1L, ifelse(clase_ternaria == "BAJA+2", 1.0000002, ifelse(clase_ternaria == "BAJA+1", 1.0000001, 1.0))],
  free_raw_data = FALSE
)


In [169]:
# Aqui comienza la configuracion de la Bayesian Optimization
funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar, # la funcion que voy a maximizar
  minimize = FALSE, # estoy Maximizando la ganancia
  noisy = TRUE,
  par.set = PARAM$hs, # definido al comienzo del programa
  has.simple.signature = FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time = 600, # se graba cada 600 segundos
  save.file.path = kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters = PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type = "se",
  covtype = "matern3_2",
  control = list(trace = TRUE)
)


In [170]:
# inicio la optimizacion bayesiana

if (!file.exists(kbayesiana)) {
  run <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else {
  run <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.



20250531 162431	binary	custom	TRUE	TRUE	FALSE	-100	31	65	TRUE	823819	0.261234506171507	61	0.545160850303364	3929	9057	5.42852127605607	57675000	1
20250531 162437	binary	custom	TRUE	TRUE	FALSE	-100	31	65	TRUE	823819	0.261234506171507	61	0.545160850303364	3929	9057	5.42852127605607	57675000	1
20250531 162517	binary	custom	TRUE	TRUE	FALSE	-100	31	225	TRUE	823819	0.142345358642245	556	0.508779525241698	4739	11126	0.726382536676996	59865000	2
20250531 162527	binary	custom	TRUE	TRUE	FALSE	-100	31	225	TRUE	823819	0.142345358642245	556	0.508779525241698	4739	11126	0.726382536676996	59865000	2
20250531 162549	binary	custom	TRUE	TRUE	FALSE	-100	31	143	TRUE	823819	0.187249453576806	286	0.797090456905426	5540	12712	4.681957731258	59910000	3
20250531 162554	binary	custom	TRUE	TRUE	FALSE	-100	31	143	TRUE	823819	0.187249453576806	286	0.797090456905426	5540	12712	4.681957731258	59910000	3
20250531 162613	binary	custom	TRUE	TRUE	FALSE	-100	31	89	TRUE	823819	0.191390381330663	408	0.718061613789177	1773	

[mbo] 0: learning_rate=0.261; num_leaves=61; feature_fraction=0.545; min_data_in_leaf=3929; envios=9057; lambda_l1=5.43 : y = 5.77e+07 : 26.7 secs : initdesign

[mbo] 0: learning_rate=0.142; num_leaves=556; feature_fraction=0.509; min_data_in_leaf=4739; envios=11126; lambda_l1=0.726 : y = 5.99e+07 : 50.0 secs : initdesign

[mbo] 0: learning_rate=0.187; num_leaves=286; feature_fraction=0.797; min_data_in_leaf=5540; envios=12712; lambda_l1=4.68 : y = 5.99e+07 : 26.6 secs : initdesign

[mbo] 0: learning_rate=0.191; num_leaves=408; feature_fraction=0.718; min_data_in_leaf=1773; envios=7812; lambda_l1=7.08 : y = 5.72e+07 : 19.1 secs : initdesign

[mbo] 0: learning_rate=0.105; num_leaves=725; feature_fraction=0.443; min_data_in_leaf=6367; envios=9428; lambda_l1=3.74 : y = 5.81e+07 : 51.0 secs : initdesign

[mbo] 0: learning_rate=0.239; num_leaves=812; feature_fraction=0.693; min_data_in_leaf=5848; envios=8682; lambda_l1=3.17 : y = 5.7e+07 : 15.8 secs : initdesign

[mbo] 0: learning_rate=0.12

20250531 164048	binary	custom	TRUE	TRUE	FALSE	-100	31	112	TRUE	823819	0.145576331795736	739	0.143953326320913	946	12366	9.99977004293158	58305000	25


[mbo] 1: learning_rate=0.146; num_leaves=739; feature_fraction=0.144; min_data_in_leaf=946; envios=12366; lambda_l1=10 : y = 5.83e+07 : 23.4 secs : infill_ei



...

[mbo] 498: learning_rate=0.0442; num_leaves=196; feature_fraction=0.685; min_data_in_leaf=1307; envios=13052; lambda_l1=1.65 : y = 5.98e+07 : 48.3 secs : infill_ei



20250601 073358	binary	custom	TRUE	TRUE	FALSE	-100	31	410	TRUE	823819	0.017813503529445	263	0.680011984814974	724	13113	0.749608115608192	62550000	523


[mbo] 499: learning_rate=0.0178; num_leaves=263; feature_fraction=0.68; min_data_in_leaf=724; envios=13113; lambda_l1=0.75 : y = 6.26e+07 : 104.6 secs : infill_ei



20250601 073727	binary	custom	TRUE	TRUE	FALSE	-100	31	1043	TRUE	823819	0.015643233276687	957	0.811480454192812	664	12608	2.42682985127488	62145000	524


[mbo] 500: learning_rate=0.0156; num_leaves=957; feature_fraction=0.811; min_data_in_leaf=664; envios=12608; lambda_l1=2.43 : y = 6.21e+07 : 189.9 secs : infill_ei

Saved the final state in the file 5431.RDATA





---

